<font color="#CC3D3D"><p>
# [Competition] Building a `LightGBM` Model with `Pipeline+Optuna`

In [1]:
LGBM_VERSION = 2.0   # submission 화일명에 사용
NFOLDS = 10
SEED = 22
TIMEOUT = 180

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import TransformedTargetRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from category_encoders import CatBoostEncoder
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV
from optuna.integration.lightgbm import LightGBMTunerCV
from scipy.sparse import csr_matrix
import re
import shap

#### Load data

In [18]:
X_train = pd.read_csv('X_train_fe_t11.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test_fe_t11.csv', encoding='cp949')
test_id = pd.read_csv('X_test.csv', encoding='cp949').ID
X_columns = X_train.columns

#### 수치형/범주형 피처 분리

In [19]:
numeric_features = [ '대학성적', '근무개월', '근무개월log', '대학랭킹', '출신대학수치', '출신대학수치*대학성적',
       '직무태그개수','대학성적0.1+근무개월', '출신대학_근무개월_sum', '출신대학_근무개월_mean', '출신대학_근무개월_variance',
       '출신대학_근무개월_standard_deviation', '직종_근무개월_sum', '직종_근무개월_mean',
       '직종_근무개월_variance', '직종_근무개월_standard_deviation', '세부직종_근무개월_sum',
       '세부직종_근무개월_mean', '세부직종_근무개월_variance', '세부직종_근무개월_standard_deviation',
       '근무지역_1_근무개월_sum', '근무지역_1_근무개월_mean', '근무지역_1_근무개월_variance',
       '근무지역_1_근무개월_standard_deviation', '근무지역_2_근무개월_sum', '근무지역_2_근무개월_mean',
       '근무지역_2_근무개월_variance', '근무지역_2_근무개월_standard_deviation',
       '근무지역_3_근무개월_sum', '근무지역_3_근무개월_mean', '근무지역_3_근무개월_variance',
       '근무지역_3_근무개월_standard_deviation', '마지막근무형태_근무개월_sum',
       '마지막근무형태_근무개월_mean', '마지막근무형태_근무개월_variance',
       '마지막근무형태_근무개월_standard_deviation', '직무태그_1_근무개월_sum',
       '직무태그_1_근무개월_mean', '직무태그_1_근무개월_variance',
       '직무태그_1_근무개월_standard_deviation', '대학성적_근무개월_sum', '대학성적_근무개월_mean',
       '대학성적_근무개월_variance', '대학성적_근무개월_standard_deviation', '대학전공_근무개월_sum',
       '대학전공_근무개월_mean', '대학전공_근무개월_variance', '대학전공_근무개월_standard_deviation',
       '어학시험_근무개월_sum', '어학시험_근무개월_mean', '어학시험_근무개월_variance',
       '어학시험_근무개월_standard_deviation','수치형통합','직종_세부직종_근무개월_sum', '직종_세부직종_근무개월_mean',
       '직종_세부직종_근무개월_variance', '직종_세부직종_근무개월_standard_deviation',
       '직종_근무지역_1_근무개월_sum', '직종_근무지역_1_근무개월_mean', '직종_근무지역_1_근무개월_variance','직종_근무지역_1_근무개월_standard_deviation', '직종_근무지역_2_근무개월_sum',
       '직종_근무지역_2_근무개월_mean', '직종_근무지역_2_근무개월_variance',
       '직종_근무지역_2_근무개월_standard_deviation', '직종_근무지역_3_근무개월_sum',
       '직종_근무지역_3_근무개월_mean', '직종_근무지역_3_근무개월_variance',
       '직종_근무지역_3_근무개월_standard_deviation', '직종_마지막근무형태_근무개월_sum',
       '직종_마지막근무형태_근무개월_mean', '직종_마지막근무형태_근무개월_variance',
       '직종_마지막근무형태_근무개월_standard_deviation', '직종_직무태그_1_근무개월_sum',
       '직종_직무태그_1_근무개월_mean', '직종_직무태그_1_근무개월_variance',
       '직종_직무태그_1_근무개월_standard_deviation', '직종_대학성적_근무개월_sum',
       '직종_대학성적_근무개월_mean', '직종_대학성적_근무개월_variance',
       '직종_대학성적_근무개월_standard_deviation', '세부직종_근무지역_1_근무개월_sum',
       '세부직종_근무지역_1_근무개월_mean', '세부직종_근무지역_1_근무개월_variance',
       '세부직종_근무지역_1_근무개월_standard_deviation', '세부직종_근무지역_2_근무개월_sum',
       '세부직종_근무지역_2_근무개월_mean', '세부직종_근무지역_2_근무개월_variance',
       '세부직종_근무지역_2_근무개월_standard_deviation', '세부직종_근무지역_3_근무개월_sum',
       '세부직종_근무지역_3_근무개월_mean', '세부직종_근무지역_3_근무개월_variance',
       '세부직종_근무지역_3_근무개월_standard_deviation', '세부직종_마지막근무형태_근무개월_sum',
       '세부직종_마지막근무형태_근무개월_mean', '세부직종_마지막근무형태_근무개월_variance',
       '세부직종_마지막근무형태_근무개월_standard_deviation', '근무지역_1_근무지역_2_근무개월_sum',
       '근무지역_1_근무지역_2_근무개월_mean', '근무지역_1_근무지역_2_근무개월_variance',
       '근무지역_1_근무지역_2_근무개월_standard_deviation', '근무지역_1_근무지역_3_근무개월_sum',
       '근무지역_1_근무지역_3_근무개월_mean', '근무지역_1_근무지역_3_근무개월_variance',
       '근무지역_1_근무지역_3_근무개월_standard_deviation', '근무지역_2_근무지역_3_근무개월_sum',
       '근무지역_2_근무지역_3_근무개월_mean', '근무지역_2_근무지역_3_근무개월_variance',
       '근무지역_2_근무지역_3_근무개월_standard_deviation', '자격증_어학시험_근무개월_sum',
       '자격증_어학시험_근무개월_mean', '자격증_어학시험_근무개월_variance',
       '자격증_어학시험_근무개월_standard_deviation', '출신대학_수치형통합_sum',
       '출신대학_수치형통합_mean', '출신대학_수치형통합_variance',
       '출신대학_수치형통합_standard_deviation', '직종_수치형통합_sum', '직종_수치형통합_mean',
       '직종_수치형통합_variance', '직종_수치형통합_standard_deviation', '세부직종_수치형통합_sum',
       '세부직종_수치형통합_mean', '세부직종_수치형통합_variance',
       '세부직종_수치형통합_standard_deviation', '근무지역_1_수치형통합_sum',
       '근무지역_1_수치형통합_mean', '근무지역_1_수치형통합_variance',
       '근무지역_1_수치형통합_standard_deviation', '근무지역_2_수치형통합_sum',
       '근무지역_2_수치형통합_mean', '근무지역_2_수치형통합_variance',
       '근무지역_2_수치형통합_standard_deviation', '근무지역_3_수치형통합_sum',
       '근무지역_3_수치형통합_mean', '근무지역_3_수치형통합_variance',
       '근무지역_3_수치형통합_standard_deviation', '마지막근무형태_수치형통합_sum',
       '마지막근무형태_수치형통합_mean', '마지막근무형태_수치형통합_variance',
       '마지막근무형태_수치형통합_standard_deviation', '직무태그_1_수치형통합_sum',
       '직무태그_1_수치형통합_mean', '직무태그_1_수치형통합_variance',
       '직무태그_1_수치형통합_standard_deviation', '대학성적_수치형통합_sum', '대학성적_수치형통합_mean',
       '대학성적_수치형통합_variance', '대학성적_수치형통합_standard_deviation',
       '대학전공_수치형통합_sum', '대학전공_수치형통합_mean', '대학전공_수치형통합_variance',
       '대학전공_수치형통합_standard_deviation', '어학시험_수치형통합_sum', '어학시험_수치형통합_mean',
       '어학시험_수치형통합_variance', '어학시험_수치형통합_standard_deviation',
       '직종_세부직종_수치형통합_sum', '직종_세부직종_수치형통합_mean']

categorical_features = ['직종', '세부직종', '직무태그', '근무경력', '근무형태', '근무지역', '출신대학', '대학전공', '어학시험',
       '자격증','근무지역_1', '근무지역_2', '근무지역_3', '마지막근무형태', '직무태그_1', '직무태그_2', '직무태그_3',
       '직무태그_4', '직무태그_5', '직무태그_6', '직무태그_7', '직무태그_8', '직무태그_9', '직무태그_10',
       '직무태그_11', '직무태그_12', '직무태그_13', '직무태그_14', '직무태그_15', '직무태그_16',
       '직무태그_17', '직무태그_18', '직무태그_19', '직무태그_20', '직무태그_21', '직무태그_22',
       '직무태그_23', '직무태그_24', '직무태그_25', '직무태그_26', '직무태그_27',
        '근무경력+근무지역_3', '근무경력+출신대학', '근무경력+근무형태', '근무경력+마지막근무형태', '대학전공+어학시험',
       '대학전공+자격증', '대학전공+근무지역_1', '대학전공+근무지역_2', '대학전공+근무지역_3', '대학전공+출신대학',
       '대학전공+근무형태', '대학전공+마지막근무형태', '어학시험+자격증', '어학시험+근무지역_1', '어학시험+근무지역_2',
       '어학시험+근무지역_3', '어학시험+출신대학', '어학시험+근무형태', '어학시험+마지막근무형태', '자격증+근무지역_1',
       '자격증+근무지역_2', '자격증+근무지역_3', '자격증+출신대학', '자격증+근무형태', '자격증+마지막근무형태',
       '근무지역_1+출신대학', '근무지역_1+근무형태', '근무지역_1+마지막근무형태', '근무지역_2+출신대학',
       '근무지역_2+근무형태', '근무지역_2+마지막근무형태', '근무지역_3+출신대학', '근무지역_3+근무형태',
       '근무지역_3+마지막근무형태', '근무형태+출신대학', '마지막근무형태+출신대학',
        "('직종', '세부직종', '직무태그')", "('직종', '세부직종', '근무경력')",
       "('직종', '세부직종', '근무형태')", "('직종', '세부직종', '출신대학')",
       "('직종', '세부직종', '대학전공')", "('직종', '세부직종', '어학시험')",
       "('직종', '세부직종', '자격증')", "('직종', '직무태그', '근무경력')",
       "('직종', '직무태그', '근무형태')", "('직종', '직무태그', '출신대학')",
       "('직종', '직무태그', '대학전공')", "('직종', '직무태그', '어학시험')",
       "('직종', '직무태그', '자격증')", "('직종', '근무경력', '근무형태')",
       "('직종', '근무경력', '출신대학')", "('직종', '근무경력', '대학전공')",
       "('직종', '근무경력', '어학시험')", "('직종', '근무경력', '자격증')",
       "('직종', '근무형태', '출신대학')", "('직종', '근무형태', '대학전공')",
       "('직종', '근무형태', '어학시험')", "('직종', '근무형태', '자격증')",
       "('직종', '출신대학', '대학전공')", "('직종', '출신대학', '어학시험')",
       "('직종', '출신대학', '자격증')", "('직종', '대학전공', '어학시험')",
       "('직종', '대학전공', '자격증')", "('직종', '어학시험', '자격증')",
       "('세부직종', '직무태그', '근무경력')", "('세부직종', '직무태그', '근무형태')",
       "('세부직종', '직무태그', '출신대학')", "('세부직종', '직무태그', '대학전공')",
       "('세부직종', '직무태그', '어학시험')", "('세부직종', '직무태그', '자격증')",
       "('세부직종', '근무경력', '근무형태')", "('세부직종', '근무경력', '출신대학')",
       "('세부직종', '근무경력', '대학전공')", "('세부직종', '근무경력', '어학시험')",
       "('세부직종', '근무경력', '자격증')", "('세부직종', '근무형태', '출신대학')",
       "('세부직종', '근무형태', '대학전공')", "('세부직종', '근무형태', '어학시험')",
       "('세부직종', '근무형태', '자격증')", "('세부직종', '출신대학', '대학전공')",
       "('세부직종', '출신대학', '어학시험')", "('세부직종', '출신대학', '자격증')",
       "('세부직종', '대학전공', '어학시험')", "('세부직종', '대학전공', '자격증')",
       "('세부직종', '어학시험', '자격증')", "('직무태그', '근무경력', '근무형태')",
       "('직무태그', '근무경력', '출신대학')", "('직무태그', '근무경력', '대학전공')",
       "('직무태그', '근무경력', '어학시험')", "('직무태그', '근무경력', '자격증')",
       "('직무태그', '근무형태', '출신대학')", "('직무태그', '근무형태', '대학전공')",
       "('직무태그', '근무형태', '어학시험')", "('직무태그', '근무형태', '자격증')",
       "('직무태그', '출신대학', '대학전공')", "('직무태그', '출신대학', '어학시험')",
       "('직무태그', '출신대학', '자격증')", "('직무태그', '대학전공', '어학시험')",
       "('직무태그', '대학전공', '자격증')", "('직무태그', '어학시험', '자격증')",
       "('근무경력', '근무형태', '출신대학')", "('근무경력', '근무형태', '대학전공')",
       "('근무경력', '근무형태', '어학시험')", "('근무경력', '근무형태', '자격증')",
       "('근무경력', '출신대학', '대학전공')", "('근무경력', '출신대학', '어학시험')",
       "('근무경력', '출신대학', '자격증')", "('근무경력', '대학전공', '어학시험')",
       "('근무경력', '대학전공', '자격증')", "('근무경력', '어학시험', '자격증')",
       "('근무형태', '출신대학', '대학전공')", "('근무형태', '출신대학', '어학시험')",
       "('근무형태', '출신대학', '자격증')", "('근무형태', '대학전공', '어학시험')",
       "('근무형태', '대학전공', '자격증')", "('근무형태', '어학시험', '자격증')",
       "('출신대학', '대학전공', '어학시험')", "('출신대학', '대학전공', '자격증')",
       "('출신대학', '어학시험', '자격증')", "('대학전공', '어학시험', '자격증')",
       "('직종', '세부직종', '근무지역_1')", "('직종', '직무태그', '근무지역_1')",
       "('직종', '근무경력', '근무지역_1')", "('직종', '근무형태', '근무지역_1')",
       "('직종', '출신대학', '근무지역_1')", "('직종', '대학전공', '근무지역_1')",
       "('직종', '어학시험', '근무지역_1')", "('직종', '자격증', '근무지역_1')",
       "('세부직종', '직무태그', '근무지역_1')", "('세부직종', '근무경력', '근무지역_1')",
       "('세부직종', '근무형태', '근무지역_1')", "('세부직종', '출신대학', '근무지역_1')",
       "('세부직종', '대학전공', '근무지역_1')", "('세부직종', '어학시험', '근무지역_1')",
       "('세부직종', '자격증', '근무지역_1')", "('직무태그', '근무경력', '근무지역_1')",
       "('직무태그', '근무형태', '근무지역_1')", "('직무태그', '출신대학', '근무지역_1')",
       "('직무태그', '대학전공', '근무지역_1')", "('직무태그', '어학시험', '근무지역_1')",
       "('직무태그', '자격증', '근무지역_1')", "('근무경력', '근무형태', '근무지역_1')",
       "('근무경력', '출신대학', '근무지역_1')", "('근무경력', '대학전공', '근무지역_1')",
       "('근무경력', '어학시험', '근무지역_1')", "('근무경력', '자격증', '근무지역_1')",
       "('근무형태', '출신대학', '근무지역_1')", "('근무형태', '대학전공', '근무지역_1')",
       "('근무형태', '어학시험', '근무지역_1')", "('근무형태', '자격증', '근무지역_1')",
       "('출신대학', '대학전공', '근무지역_1')", "('출신대학', '어학시험', '근무지역_1')",
       "('출신대학', '자격증', '근무지역_1')", "('대학전공', '어학시험', '근무지역_1')",
       "('대학전공', '자격증', '근무지역_1')", "('어학시험', '자격증', '근무지역_1')",
       "('직종', '세부직종', '근무지역_2')", "('직종', '직무태그', '근무지역_2')",
       "('직종', '근무경력', '근무지역_2')", "('직종', '근무형태', '근무지역_2')",
       "('직종', '출신대학', '근무지역_2')", "('직종', '대학전공', '근무지역_2')",
       "('직종', '어학시험', '근무지역_2')", "('직종', '자격증', '근무지역_2')",
       "('세부직종', '직무태그', '근무지역_2')", "('세부직종', '근무경력', '근무지역_2')",
       "('세부직종', '근무형태', '근무지역_2')", "('세부직종', '출신대학', '근무지역_2')",
       "('세부직종', '대학전공', '근무지역_2')", "('세부직종', '어학시험', '근무지역_2')",
       "('세부직종', '자격증', '근무지역_2')", "('직무태그', '근무경력', '근무지역_2')",
       "('직무태그', '근무형태', '근무지역_2')", "('직무태그', '출신대학', '근무지역_2')",
       "('직무태그', '대학전공', '근무지역_2')", "('직무태그', '어학시험', '근무지역_2')",
       "('직무태그', '자격증', '근무지역_2')", "('근무경력', '근무형태', '근무지역_2')",
       "('근무경력', '출신대학', '근무지역_2')", "('근무경력', '대학전공', '근무지역_2')",
       "('근무경력', '어학시험', '근무지역_2')", "('근무경력', '자격증', '근무지역_2')",
       "('근무형태', '출신대학', '근무지역_2')", "('근무형태', '대학전공', '근무지역_2')",
       "('근무형태', '어학시험', '근무지역_2')", "('근무형태', '자격증', '근무지역_2')",
       "('출신대학', '대학전공', '근무지역_2')", "('출신대학', '어학시험', '근무지역_2')",
       "('출신대학', '자격증', '근무지역_2')", "('대학전공', '어학시험', '근무지역_2')",
       "('대학전공', '자격증', '근무지역_2')", "('어학시험', '자격증', '근무지역_2')",
       "('직종', '세부직종', '근무지역_3')", "('직종', '직무태그', '근무지역_3')",
       "('직종', '근무경력', '근무지역_3')", "('직종', '근무형태', '근무지역_3')",
       "('직종', '출신대학', '근무지역_3')", "('직종', '대학전공', '근무지역_3')",
       "('직종', '어학시험', '근무지역_3')", "('직종', '자격증', '근무지역_3')",
       "('세부직종', '직무태그', '근무지역_3')", "('세부직종', '근무경력', '근무지역_3')",
       "('세부직종', '근무형태', '근무지역_3')", "('세부직종', '출신대학', '근무지역_3')",
       "('세부직종', '대학전공', '근무지역_3')", "('세부직종', '어학시험', '근무지역_3')",
       "('세부직종', '자격증', '근무지역_3')", "('직무태그', '근무경력', '근무지역_3')",
       "('직무태그', '근무형태', '근무지역_3')", "('직무태그', '출신대학', '근무지역_3')",
       "('직무태그', '대학전공', '근무지역_3')", "('직무태그', '어학시험', '근무지역_3')",
       "('직무태그', '자격증', '근무지역_3')", "('근무경력', '근무형태', '근무지역_3')",
       "('근무경력', '출신대학', '근무지역_3')", "('근무경력', '대학전공', '근무지역_3')",
       "('근무경력', '어학시험', '근무지역_3')", "('근무경력', '자격증', '근무지역_3')",
       "('근무형태', '출신대학', '근무지역_3')", "('근무형태', '대학전공', '근무지역_3')",
       "('근무형태', '어학시험', '근무지역_3')", "('근무형태', '자격증', '근무지역_3')",
       "('출신대학', '대학전공', '근무지역_3')", "('출신대학', '어학시험', '근무지역_3')",
       "('출신대학', '자격증', '근무지역_3')", "('대학전공', '어학시험', '근무지역_3')",
       "('대학전공', '자격증', '근무지역_3')", "('어학시험', '자격증', '근무지역_3')"
       ]
pca_features = []

X_train = X_train[numeric_features+categorical_features+binary_features+pca_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features+binary_features+pca_features]

####  파이프라인 구축

In [20]:
def remove_outlier(X, q=0.05):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

In [21]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})), 
        ("scaler", PowerTransformer()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int)),
    ]
)
    
binary_transformer = Pipeline(
    steps=[
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
    ]
)

pca_transformer = Pipeline(
    steps=[
        ("impuer", FunctionTransformer(lambda x: x.fillna('없음'))),      
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
        ("dense", FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
        ("pca", PCA(n_components=100)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bin1", binary_transformer, binary_features[0]), 
        ("bin2", binary_transformer, binary_features[1]),
        ("pca1", pca_transformer, pca_features[0]), 
   ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
        ("selector", SelectPercentile(percentile=100)),
    ]
)

set_config(display="diagram")
preprocessor

Pipeline(steps=[('column',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('outlier',
                                                                   FunctionTransformer(func=<function remove_outlier at 0x0000021EB2768670>,
                                                                                       kw_args={'q': 0.05})),
                                                                  ('scaler',
                                                                   PowerTransformer())]),
                                                  ['대학성적', '근무개월', '출신대학*대학성적',
                                                   '직무수', '개월2', '개월log']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='...
                                                                   FunctionTransformer(func=<function <lambda> at 0x0000021EB2768430>)),
                                                                  ('corpus',
                                                                   FunctionTransformer(func=<function <lambda> at 0x0000021EB2768280>)),
                                                                  ('BoW',
                                                                   CountVectorizer()),
                                                                  ('dense',
                                                                   FunctionTransformer(accept_sparse=True,
                                                                                       func=<function <lambda> at 0x0000021EB27681F0>)),
                                                                  ('pca',
                                                                   PCA(n_components=100))]),
                                                  '직무태그')])),
                ('selector', SelectPercentile(percentile=100))])

#### SHAP

In [22]:
X_train = preprocessor.fit_transform(X_train, y_train)
X_test = preprocessor.transform(X_test)

In [23]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [31]:
model = LGBMRegressor(random_state=SEED).fit(X_train, y_train)
X_importance = X_test

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_importance)

shap_sum = np.abs(shap_values).mean(axis=0)

In [33]:
importance_df = pd.DataFrame([X_train.columns, shap_sum]).T
importance_df.columns = ['feature_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)

#feature_name에 NAN 나오는 거 정상임

In [34]:
features_selected = importance_df.query('shap_importance > 0').index

In [35]:
features_selected

Int64Index([  1,  21,   6,  20,   2,  18,  11, 312,   7,  17,
            ...
            216, 200, 177, 149,  33, 122, 213, 294, 183,   3],
           dtype='int64', length=164)

In [36]:
X_train = X_train.iloc[:, features_selected].to_numpy()
X_test = X_test.iloc[:, features_selected]

#### LGBM Baseline 성능 확인

In [37]:
sscv = ShuffleSplit(test_size=.3334, n_splits=NFOLDS, random_state=SEED) # Public LB 사이즈와 동일하게 평가데이터 사이즈 설정 
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=sscv)

print("Default LGBM CV scores: ", np.sqrt(-1*scores))
print("Default LGBM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

Default LGBM CV scores:  [830.02281306 861.07047579 850.9670844  829.90335873 823.48421673
 834.03860814 822.21032981 832.2114992  816.6772845  833.59632675]
Default LGBM CV mean = 833.51 with std = 145.80


#### LGBM 하이퍼파라미터 최적화
- 1단계) 전처리 파라미터 최적값 탐색: OptunaSearchCV를 통해 전처리 관련 최적의 파라미터을 찾음 (LGBM은 No Tuning)

In [ ]:
%%time

param_distributions = {
    "preprocessor__column__num__imputer__strategy": CategoricalDistribution(["mean","median"]),
    "preprocessor__column__num__outlier__kw_args": CategoricalDistribution([{'q':0.01},{'q':0.05},{'q':0.1}]),
    "preprocessor__column__pca1__pca__n_components": IntDistribution(100,500,step=100),  
    "preprocessor__selector__percentile": IntDistribution(50,100,step=10),  
}

optuna_search = OptunaSearchCV(model, 
                               param_distributions, 
                               cv=sscv, scoring='neg_mean_squared_error', 
                               n_trials=3,
                               timeout=TIMEOUT,   # 튜닝 허용 최대시간(초단위)
                               study=optuna.create_study(sampler=optuna.samplers.TPESampler(seed=100), direction="maximize"))

optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search.fit(X_train, y_train)

print(f"Best params: {optuna_search.best_params_}")
print("Best score: %.2f" % (-1*optuna_search.best_score_)**0.5)

- 2단계) 전처리 최적화 수행: 최적의 파라미터 값으로 파이프라인 재설정

In [ ]:
# 최적값으로 파이프라인 재설정
model.set_params(**optuna_search.best_params_)

# 전처리 파이프라인만 수행
X_train = preprocessor.fit_transform(X_train, y_train)
X_test = preprocessor.transform(X_test)

- 3단계) LGBM 모형 최적화: Optuna의 LightGBMTunerCV을 통한 하이퍼파라미터 최적화

In [ ]:
tuner = LightGBMTunerCV(
    params={
        "objective": "regression",   # 지도학습 유형(regression/binary/multiclass)
        "metric": "rmse",
        "verbosity": -1,             # 진행과정 출력안함
        "boosting_type": "gbdt",     # 실행하고자 하는 알고리즘 유형(gdbt/rf/dart/goss)
        "seed": 0,
    },
    train_set=lgb.Dataset(X_train, y_train), # LGBM 데이터셋으로 변환
    nfold=5,
    num_boost_round=200,                     # boosting 반복횟수
    callbacks=[lgb.early_stopping(100)],     # 모든 정지 라운드 동안 validation 성능이 개선되지 않으면 조기종료
    time_budget=TIMEOUT,                     # 튜닝 허용 최대시간(초단위)
    optuna_seed=0,
)

tuner.run()

In [ ]:
print(f"\nBest params: {tuner.best_params}")
print(f"\nBest score: {tuner.best_score:.2f}")

#### Submission 생성

In [ ]:
# 최적화된 하이퍼파라미터로 OOF를 수행하여 최종 LGBM 모형 생성
models = cross_validate(LGBMRegressor(**tuner.best_params), # 최적화된 hyperparameter 사용
                        X_train, y_train, 
                        cv=sscv, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

scores = models['test_score']
print("\nTuned LGBM CV scores: ", np.sqrt(-1*scores))
print("Tuned LGBM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

In [ ]:
# submission 화일 생성
filename = f'lgbm_{LGBM_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)

<font color="#CC3D3D"><p>
# End